# Kong: Performance Evaluation

In this notebook, we report on some experimental results obtained with Kong.

### Setup Analysis

Import librairies.

In [ ]:
%matplotlib inline

import math
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

Path to data.

In [ ]:
path_data = 'OUTPUTS/'

Settings.

In [ ]:
TIMEOUT = 3600

## Load Data

Reductions data frame.

In [ ]:
# Read 'reductions.csv'
df_reductions = pd.read_csv(path_data + 'reductions.csv')
df_reductions

Complete matrix computations data frame (global timeout of 3,600 seconds).

In [ ]:
# Read 'complete_computations.csv'
df_complete_computations = pd.read_csv(path_data + 'complete_computations.csv')
df_complete_computations

Partial matrix computations data frame (timeout of 60 seconds for the BDD exploration and global timeout of 3,600 seconds).

In [ ]:
# Read 'partial_computations.csv'
df_partial_computations = pd.read_csv(path_data + 'partial_computations.csv')
df_partial_computations

## Tool Confidence rate

### Reliability

Complete matrices.

In [ ]:
reliability_complete_matrices = (1 - df_complete_computations.query('CORRECTNESS == False').shape[0] / df_complete_computations.shape[0]) * 100
print('{}%'.format(math.floor(reliability_complete_matrices)))

Partial matrices.

In [ ]:
reliability_partial_matrices = (1 - df_partial_computations.query('CORRECTNESS == False').shape[0] / df_partial_computations.shape[0]) * 100
print('{}%'.format(math.floor(reliability_partial_matrices)))

### Correct Matrices

Number of correct and complete matrices.

In [ ]:
df_complete_computations.query('CORRECTNESS == True and TIME_KONG == TIME_KONG').shape[0]

### Summary Table

Computed matrices using Kong.

In [ ]:
computed_matrices_using_Kong = df_complete_computations.query('TIME_KONG == TIME_KONG').shape[0]
computed_matrices_using_Kong

Computed matrices using caesar.bdd.

In [ ]:
computed_matrices_using_caesar = df_complete_computations.query('TIME_CAESAR == TIME_CAESAR').shape[0]
computed_matrices_using_caesar

In [ ]:
summary = {'Reliability': [reliability_complete_matrices, np.nan], 'Computed Matrices': [computed_matrices_using_Kong, computed_matrices_using_caesar]}
pd.DataFrame(data=summary, index=['Kong', 'Caesar'])

## Analysis

Performance evalutation of the *polyhedral abstraction* approach.

### Reduction Ratio 

Reduction ratio among instances.

In [ ]:
# Count instances with the same reduction ratio
ratio_frequency = df_reductions['RATIO'].value_counts().sort_index(ascending=False)

# Cumulative data frame
df_ratio_frequency = pd.DataFrame({'INSTANCES': ratio_frequency.values}).cumsum()
df_ratio_frequency['RATIO'] = ratio_frequency.index

# Add row '0'
df_first_row = pd.DataFrame([{'INSTANCES': 1, 'RATIO': df_ratio_frequency['RATIO'][0]}])
df_ratio_frequency = pd.concat([df_first_row, df_ratio_frequency])

# Draw instances reduction ratio
df_ratio_frequency.plot.area(x='INSTANCES', xlim=(0,df_reductions.shape[0]), ylim=(0,100), color='cornflowerblue', figsize=(13,3), legend='')
plt.xlabel('Number of instances', fontsize=13)
plt.ylabel('Reduction ratio (%)', fontsize=13)
plt.savefig('reductions.png', bbox_inches = 'tight')
plt.show()

### General Performance Overview

In [ ]:
# Get computations with the corresponding reduction ratio
df_computations_with_ratio = df_complete_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE')

Median reduction time.

In [ ]:
df_computations_with_ratio['TIME'].median()

Mean reduction time.

In [ ]:
df_computations_with_ratio['TIME'].mean()

In [ ]:
def information_per_reduction_range(ratio_min, ratio_max):
    """ Return summary information for a given reduction range.
    """
    df = df_computations_with_ratio.query('RATIO >= {} and RATIO <= {}'.format(ratio_min, ratio_max))

    reduction_range = '{}-{}%'.format(ratio_min, ratio_max)

    number_instances = df[['INSTANCE']].drop_duplicates().shape[0]
    computed_matrices_using_Kong = df.query('TIME_KONG == TIME_KONG').shape[0]
    computed_matrices_using_caesar = df.query('TIME_CAESAR == TIME_CAESAR').shape[0]
    gain = (1 - computed_matrices_using_caesar / computed_matrices_using_Kong) * 100

    return [reduction_range, number_instances, computed_matrices_using_Kong, computed_matrices_using_caesar, gain]

Summary table.

In [ ]:
performance_overview = pd.DataFrame([information_per_reduction_range(ratio_min, ratio_max) for ratio_min, ratio_max in [[30, 100], [30, 70], [60, 99], [100,100]]], columns=['Reduction Ratio', 'Number of Instances', 'Computed Matrices using Kong', 'Computed Matrices using Caesar.bdd', 'Gain (%)'])
performance_overview.set_index('Reduction Ratio')

### Matrix Computation Times: With VS Without Reduction

Comparison of the number of computed matrices in a limited time (3,600 seconds) between Kong and caesar.bdd.

In [ ]:
def matrix_computation_times(ratio_min, ratio_max):
    """ Plot matrix computation times with vs without reduction.
    """
    # Get computed matrices for a given reduction range (remove instances that timeout using Kong and caesar.bdd)
    df = df_complete_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and (TIME_KONG == TIME_KONG or TIME_CAESAR == TIME_CAESAR)'.format(ratio_min, ratio_max))
    
    # Replace timeout NaN values by the timeout value
    df.loc[df.TIME_KONG != df.TIME_KONG, 'TIME_KONG'] = TIMEOUT
    df.loc[df.TIME_CAESAR != df.TIME_CAESAR, 'TIME_CAESAR'] = TIMEOUT

    # Replace 0 values by 0.1
    df.loc[df.TIME_KONG == 0, 'TIME_KONG'] = 0.01
    df.loc[df.TIME_CAESAR == 0, 'TIME_CAESAR'] = 0.01

    # Get times using Kong and caesar 
    x = df['TIME_CAESAR'].to_numpy()
    y = df['TIME_KONG'].to_numpy()
     
    # Plot time with vs without reduction with a log scale
    plt.figure(figsize=(8,8))
    ax = plt.gca()
    plt.scatter(x=x, y=y, marker='+', s=100)  
    plt.plot(np.linspace(0.00, TIMEOUT), np.linspace(0.00, TIMEOUT), color='black', linestyle='--', lw=2, scalex=False, scaley=False)
    plt.plot(np.linspace(0.00, TIMEOUT), 0.1 * np.linspace(0.00, TIMEOUT), color='orange', linestyle=':', lw=2, scalex=False, scaley=False)
    plt.plot(np.linspace(0.00, TIMEOUT), 0.01 * np.linspace(0.00, TIMEOUT), color='red', linestyle=':', lw=2, scalex=False, scaley=False)
    ax.set_xscale('log')
    ax.set_yscale('log')
    plt.xlabel('Computation time without reduction (s)', fontsize=13)
    plt.ylabel('Computation time with reduction (s)', fontsize=13)
    plt.savefig("time_{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

    # Plot the number of instances and the number of computed matrices with and without reduction
    plt.figure(figsize=(1,8))
    plt.bar([0], [df_complete_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {}'.format(ratio_min, ratio_max)).shape[0]], color='orange', label='All instances')
    plt.bar([0], [df_complete_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and TIME_KONG < {}'.format(ratio_min, ratio_max, TIMEOUT)).shape[0]], label='Computed matrices with reduction')
    plt.bar([0], [df_complete_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and TIME_CAESAR < {}'.format(ratio_min, ratio_max, TIMEOUT)).shape[0]], color='cyan', label='Computed matrices without reduction')
    plt.legend(loc='upper left', fontsize=13)
    ax = plt.gca()
    ax.axes.xaxis.set_visible(False)
    plt.xlim([0, 0.1])
    plt.savefig("bar_{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

In [ ]:
matrix_computation_times(50, 100)

### Partial Concurrency Matrices: With VS Without Reduction

In [ ]:
# Drop instances that take more than 1000 seconds using Kong or caesar.bdd (outliers).
df_partial_computations_without_outliers = df_partial_computations.query('TIME_KONG < 1000 and TIME_CAESAR < 1000')

Comparison of the filling ratio with and without reduction (60 seconds of timeout for the BDD exploration).

In [ ]:
def filling_ratio(ratio_min, ratio_max):
    """ Plot the filling ratio of the matrices with vs without reduction.
    """
    # Get computed relations for a given reduction range (remove instances that timeout or that are fully computed with both tools)
    df = df_partial_computations_without_outliers.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and (NUMBER_RELATIONS_KONG > 0 or NUMBER_RELATIONS_CAESAR > 0) and (NUMBER_RELATIONS_KONG < RELATION_SIZE or NUMBER_RELATIONS_CAESAR < RELATION_SIZE)'.format(ratio_min, ratio_max))
    
    # Get filling ratios using Kong and caesar.bdd
    x = 100 * df['NUMBER_RELATIONS_CAESAR'].to_numpy() / df['RELATION_SIZE'].to_numpy()
    y = 100 * df['NUMBER_RELATIONS_KONG'].to_numpy() / df['RELATION_SIZE'].to_numpy()

    # Plot ratios with vs without reduction
    plt.figure(figsize=(8,8))
    ax = plt.gca()
    plt.scatter(x=x, y=y, marker='+', s=100)  
    plt.plot(np.linspace(0, 500), np.linspace(0, 500), color='black', linestyle='--', lw=2, scalex=False, scaley=False)
    plt.xlim(-0.1,101)
    plt.ylim(-0.1,101)
    plt.xlabel('Filling ratio of the matrix without reduction (%)', fontsize=13)
    plt.ylabel('Filling ratio of the matrix with reduction (%)', fontsize=13)
    plt.savefig("filling_ratio{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

In [ ]:
filling_ratio(50, 100)

Median computation time per instance with Kong.

In [ ]:
df_partial_computations_without_outliers['TIME_KONG'].median()

Mean computation time per instance with Kong.

In [ ]:
df_partial_computations_without_outliers['TIME_KONG'].mean()

Median computation time per instance with caesar.bdd.

In [ ]:
df_partial_computations_without_outliers['TIME_CAESAR'].median()

Median computation time per instance with caesar.bdd.

In [ ]:
df_partial_computations_without_outliers['TIME_CAESAR'].mean()

Total computation times per instance between Kong and caesar.bdd.

In [ ]:
def partial_computation_times(ratio_min, ratio_max):
    """ Plot computation times for partial matrices with vs without reduction.
    """
    # Get computed relations for a given reduction range
    df = df_partial_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {}'.format(ratio_min, ratio_max)).sort_values(['TIME_CAESAR', 'TIME_KONG'])
    
    # Replace NaN values by timeout value
    df.loc[df.TIME_KONG != df.TIME_KONG, 'TIME_KONG'] = TIMEOUT
    df.loc[df.TIME_CAESAR != df.TIME_CAESAR, 'TIME_CAESAR'] = TIMEOUT

    # Ignore instances that timeout using both tools
    df = df.query('TIME_KONG < {} or TIME_CAESAR < {}'.format(TIMEOUT, TIMEOUT))
    
    # Get computations times
    time_kong = df['TIME_KONG'].to_numpy()
    time_caesar = df['TIME_CAESAR'].to_numpy()

    # Plot times
    plt.figure(figsize=(10,4))
    plt.plot(time_caesar)
    plt.plot(time_kong)
    plt.xlabel('Instances', fontsize=13)
    plt.ylabel('Computation time (s)', fontsize=13)
    plt.savefig("partial_computation_times_{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

In [ ]:
partial_computation_times(50,100)

### Concurrent and Independent Places: With vs Without Reduction

Comparison of the filling ratio for concurrent places and independent places (independently) with vs without reduction.

In [ ]:
def concurrent_and_independant_places_filling_ratio(ratio_min, ratio_max):
    """ Plot the filling ratio for concurrent and independent places with vs without reduction.
    """
    # Get computed relations for a given reduction range (remove instances that timeout or that are fully computed with both tools)
    df = df_partial_computations_without_outliers.drop(['RELATION_SIZE', 'TIME_KONG', 'TIME_CAESAR', 'CORRECTNESS'], axis=1).join(df_reductions.drop(['TIME'], axis=1).set_index('INSTANCE'), on='INSTANCE').join(df_complete_computations.drop(['TIME_KONG', 'TIME_CAESAR', 'CORRECTNESS'], axis=1).set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and RELATION_SIZE > 0 and (NUMBER_RELATIONS_KONG > 0 or NUMBER_RELATIONS_CAESAR > 0) and (NUMBER_RELATIONS_KONG < RELATION_SIZE or NUMBER_RELATIONS_CAESAR < RELATION_SIZE)'.format(ratio_min, ratio_max))

    # Get filling ratios for concurrent places ('1') using Kong and caesar.bdd
    x = 100 * df['CONCURRENT_PLACES_CAESAR'].to_numpy() / df['CONCURRENT_PLACES'].to_numpy()
    y = 100 * df['CONCURRENT_PLACES_KONG'].to_numpy() / df['CONCURRENT_PLACES'].to_numpy()

    # Plot ratios with vs without reduction
    plt.figure(figsize=(8,8))
    ax = plt.gca()
    plt.scatter(x=x, y=y, marker='+', s=100)  
    plt.plot(np.linspace(0, 500), np.linspace(0, 500), color='black', linestyle='--', lw=2, scalex=False, scaley=False)
    plt.xlim(-0.1,101)
    plt.ylim(-0.1,101)
    plt.xlabel('Filling ratio of the concurrent places without reduction (%)', fontsize=13)
    plt.ylabel('Filling ratio of the concurrent places with reduction (%)', fontsize=13)
    plt.savefig("concurrent_places_{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

    # Get filling ratios for independent places ('0') using Kong and caesar.bdd
    x = 100 * (df['NUMBER_RELATIONS_CAESAR'].to_numpy() - df['CONCURRENT_PLACES_CAESAR'].to_numpy()) / (df['RELATION_SIZE'].to_numpy() - df['CONCURRENT_PLACES'].to_numpy())
    y = 100 * (df['NUMBER_RELATIONS_KONG'].to_numpy() - df['CONCURRENT_PLACES_KONG'].to_numpy()) / (df['RELATION_SIZE'].to_numpy() - df['CONCURRENT_PLACES'].to_numpy())

    # Plot filling ratios with vs without reduction
    plt.figure(figsize=(8,8))
    ax = plt.gca()
    plt.scatter(x=x, y=y, marker='+', s=100)  
    plt.plot(np.linspace(0, 500), np.linspace(0, 500), color='black', linestyle='--', lw=2, scalex=False, scaley=False)
    plt.xlim(-0.1,101)
    plt.ylim(-0.1,101)
    plt.xlabel('Filling ratio of the independent places without reduction (%)', fontsize=13)
    plt.ylabel('Filling ratio of the independent places with reduction (%)', fontsize=13)
    plt.savefig("independent_places_{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

In [ ]:
concurrent_and_independant_places_filling_ratio(50, 100)